In [40]:
from random import uniform, random
from math import e, sqrt, cos, pi
import numpy as np
import random as random
from scipy.optimize import minimize

In [77]:
class ABC():
    class Bee():
        
        def __init__(self, position) -> None:
            self.memory = position

        def get_fitness(self, position, fx):
            cost = fx(position)
            if(cost >= 0):
                return 1/1+cost
            return 1 + abs(cost)

        def find_neighbourhood(self, a, swarm, fx):
            i = random.randint(0,len(swarm)-1)
            j = random.randint(0,len(self.memory)-1)
            #neighbour = self.memory + random.randint(-a,a)*(self.memory[j] - swarm[i].memory[j])
            neighbour = minimize(fx, self.memory, method='COBYLA').x
            cost_neighbour = self.get_fitness(neighbour,fx)
            cost_memory = self.get_fitness(self.memory,fx)
            if(cost_neighbour < cost_memory):
                self.memory = neighbour
                return 1
            else:
                return 0

        def set_memory(self, position):
            self.memory = position

        def get_memory(self):
            return self.memory

    class OnlookerBee(Bee):

        def __init__(self,position) -> None:
            super().__init__(position)

        def comunicate(self, swarm, fx, a):
            population_fitness = sum([bee.get_fitness(bee.memory, fx) for bee in swarm])
            bees_probabilities = [bee.get_fitness(bee.memory, fx)/population_fitness for bee in swarm]
            bee = np.random.choice(swarm, p=bees_probabilities)
            super().set_memory(bee.memory)
            better_neighbourhood = super().find_neighbourhood(a, swarm, fx)
            if(better_neighbourhood == 1):
                if(fx(super().get_memory()) < fx(bee.memory)):
                    bee.set_memory(super().get_memory())


    class EmployedBee(Bee):

        def __init__(self,position, lower, upper) -> None:
            super().__init__(position)
            self.lower = lower
            self.upper = upper
            self.tries = 0

        def move(self, a, swarm, fx, limit):
            if(self.tries < limit):
                gain = super().find_neighbourhood(a, swarm, fx)
                if(gain == 0):
                    self.tries += 1
            else:
                self.tries = 0
                self.scout()

        def scout(self):
            super().set_memory(np.array([self.lower + random.uniform(0, 1)*(self.upper-self.lower) for _ in range(0, len(super().get_memory()))]))
    
    def __init__(self, a, dimention, num_points, bonds, numlookers) -> None:
        self.a = a
        self.num_points = num_points
        self.swarm = [self.EmployedBee(np.array([bonds[0] + random.uniform(0, 1)*(bonds[1]-bonds[0]) for _ in range(0, dimention)]), bonds[0], bonds[1]) for _ in range(0, num_points)]
        self.unlooker_bees = [self.OnlookerBee(np.ones(numlookers, dtype = int)) for _ in range(0, num_points)]
        
    def run(self,fx, num_iterations, limit):
        def find_best():
            best_cost = 0
            best_bee = list()
            for workerbee in self.swarm:
                cost_bee = -1*fx(workerbee.memory)
                if(cost_bee < best_cost):
                    best_cost = cost_bee
                    best_bee = workerbee
            for unlookerbee in self.swarm:
                cost_bee = -1*fx(unlookerbee.memory)
                if(cost_bee < best_cost):
                    best_cost = cost_bee
                    best_bee = unlookerbee
            return best_bee
        for unlookerbee in self.unlooker_bees:
            unlookerbee.comunicate(self.swarm, fx, self.a)
        best_bee = find_best()
        for _ in range(0, num_iterations):
            for workerbee in self.swarm:
                workerbee.move( self.a, self.swarm, fx, limit)
            for unlookerbee in self.unlooker_bees:
                unlookerbee.comunicate(self.swarm, fx, self.a)
            act_best = find_best()
            if(fx(act_best.memory) < fx(best_bee.memory)):
                best_bee = act_best
        return best_bee.memory

## TEST'S

### Himmelblau's function

![alt text](him.png "Title")

In [70]:
fx = lambda x : (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 -7)**2

In [71]:
abc =  ABC(a=2, dimention=2, num_points=30, bonds=[-5, 5], numlookers=30)
abc.run(fx=fx,num_iterations=200, limit=50)

array([-4.3985673 , -3.30432524])

### Ackley function

![alt text](akley.png "Title")

In [78]:
fx = lambda x : -20*e**(-0.2*sqrt(0.5*(x[0]**2 + x[1]**2)))-e**(0.5*(cos(2*pi*x[1])+cos(2*pi*x[0])))+e+20

In [79]:
abc = ABC(a=2, dimention=2, num_points=50, bonds=[-5, 5], numlookers=30)
abc.run(fx=fx,num_iterations=200, limit=20)

array([ 5.24548041e-05, -1.13681939e-04])

### Rastrigin

![alt text](ras.png "Title")

In [80]:
fx = lambda x : 10*2 + (x[0]**2 - 10*cos(2*pi*x[0])) + (x[1]**2 - 10*cos(2*pi*x[1]))

In [81]:
abc = ABC(a=2, dimention=2, num_points=50, bonds=[-5, 5], numlookers=30)
abc.run(fx=fx,num_iterations=200, limit=20)

array([-6.23173770e-05, -9.00946625e-05])